### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [28]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [29]:
X_test.head(3)

,SeniorCitizen,tenure,MonthlyCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,6,19.65,0,0,0,1,0,0,0,...,1,0,1,0,0,1,0,1,0,0
1,0,13,53.45,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,49,97.95,1,1,1,1,0,1,1,...,0,1,0,1,1,0,1,0,1,0


In [30]:
import urllib.request
import json      


def get_prediction(x):
    body = x 
    myurl = "http://127.0.0.1:5000/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [31]:
get_prediction(X_test.iloc[1].to_dict())

0.26666666666666666

In [32]:
type(X_test)

pandas.core.frame.DataFrame

In [33]:
%%time
predictions = X_test.iloc[:500].apply(lambda x: get_prediction(x.to_dict()), 1)

Wall time: 10.7 s


In [34]:
predictions

0      0.003333
1      0.266667
2      0.383333
3      0.310000
4      0.086667
         ...   
495    0.390000
496    0.376667
497    0.000000
498    0.056667
499    0.360000
Length: 500, dtype: float64

In [35]:
len(predictions)

500

In [36]:
roc_auc_score(y_score=predictions.iloc[:500].values, y_true=y_test.iloc[:500])

0.817256841808932